In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils import data
from torch import nn
import torch.nn.functional as F
from torch import nn, optim, autograd
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import pickle

def write_results_to_file(filename, data):
    with open(filename, 'wb') as handle:
        pickle.dump(data, handle, protocol=2)

def load_results(filename):
    with open(filename, 'rb') as handle:
        dataset= pickle.load(handle)
        return dataset

def append_results_to_file(filename, data):
    with open(filename, 'a+b') as handle:
        pickle.dump(data, handle, protocol=2)

In [3]:
class Dataset(data.Dataset):
    """Characterizes a dataset for PyTorch"""
    def __init__(self, data, y, t, T):
        """Initialization""" 
        
        self.T = T
        
        all_data = []
        all_label = []
        all_treatment = []
        for k in range(int(len(data)*0.8)): # maximum length is 20 len(data)
            data_window = []
            label_window = []
            for i in range(T, len(data[k])):
                data_window.append(data[k][i-T:i])
            all_data.append(data_window)
            all_label.append(y[k][T+1:])
            all_treatment.append(t[k][T:])
        
        x = np.array(all_data)
        
        y = np.array(all_label)
        
        t = np.array(all_treatment)
            
        self.length = len(x)

        x = torch.from_numpy(x)
        #self.x = x #torch.unsqueeze(x, 1)
        self.x = x #torch.unsqueeze(x, 1)
        self.y = torch.from_numpy(y)
        self.t = torch.from_numpy(t)

    def __len__(self):
        """Denotes the total number of samples"""
        return self.length

    def __getitem__(self, index):
        """Generates samples of data"""
        return self.x[index], self.y[index], self.t[index]

In [4]:
class Dataset_in_test(data.Dataset):
    """Characterizes a dataset for PyTorch"""
    def __init__(self, data, y, t, T):
        """Initialization""" 
        
        self.T = T
        
        all_data = []
        all_label = []
        all_treatment = []
        for k in range(int(len(data)*0.8),len(data)): # maximum length is 20 len(data)
            data_window = []
            label_window = []
            for i in range(T, len(data[k])):
                data_window.append(data[k][i-T:i])
            all_data.append(data_window)
            all_label.append(y[k][T+1:])
            all_treatment.append(t[k][T:])
        
        x = np.array(all_data)
        
        y = np.array(all_label)
        
        t = np.array(all_treatment)
            
        self.length = len(x)

        x = torch.from_numpy(x)
        #self.x = x #torch.unsqueeze(x, 1)
        self.x = x #torch.unsqueeze(x, 1)
        self.y = torch.from_numpy(y)
        self.t = torch.from_numpy(t)

    def __len__(self):
        """Denotes the total number of samples"""
        return self.length

    def __getitem__(self, index):
        """Generates samples of data"""
        return self.x[index], self.y[index], self.t[index]

In [5]:
class Dataset_test(data.Dataset):
    """Characterizes a dataset for PyTorch"""
    def __init__(self, data, y, t, T):
        """Initialization""" 
        
        self.T = T
        
        all_data = []
        all_label = []
        all_treatment = []
        for k in range(len(data)): # maximum length is 20 len(data)
            data_window = []
            label_window = []
            for i in range(T, len(data[k])):
                data_window.append(data[k][i-T:i])
            all_data.append(data_window)
            all_label.append(y[k][T+1:])
            all_treatment.append(t[k][T:])
        
        x = np.array(all_data)
        
        y = np.array(all_label)
        
        t = np.array(all_treatment)
            
        self.length = len(x)

        x = torch.from_numpy(x)
        #self.x = x #torch.unsqueeze(x, 1)
        self.x = x #torch.unsqueeze(x, 1)
        self.y = torch.from_numpy(y)
        self.t = torch.from_numpy(t)

    def __len__(self):
        """Denotes the total number of samples"""
        return self.length

    def __getitem__(self, index):
        """Generates samples of data"""
        return self.x[index], self.y[index], self.t[index]

In [6]:
train_envs = load_results('/Covid-dataset/causalcovid19/data_2023/train_envs.txt')
test_envs = load_results('/Covid-dataset/causalcovid19/data_2023/test_envs.txt')

In [7]:
all_train_loader = []
for key in train_envs.keys():
    dataset_train = Dataset(train_envs[key]['previous_covariates'], train_envs[key]['outcomes'],train_envs[key]['previous_treatments'], T=5)
    train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=4, shuffle=True)  
    all_train_loader.append(train_loader)


In [8]:
all_test_loader = []
for key in test_envs.keys():
    dataset_train = Dataset_test(test_envs[key]['previous_covariates'], test_envs[key]['outcomes'],test_envs[key]['previous_treatments'], T=5)
    train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=4, shuffle=True)  
    all_test_loader.append(train_loader)

In [9]:
all_test_in_loader = []
for key in train_envs.keys():
    dataset_train = Dataset_in_test(train_envs[key]['previous_covariates'], train_envs[key]['outcomes'],train_envs[key]['previous_treatments'], T=5)
    train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=4, shuffle=True)  
    all_test_in_loader.append(train_loader)

In [10]:
def report_res(u_pred,u_label):
    
    d = u_pred - u_label
    mse_f = np.mean(d**2)
    mae_f = np.mean(abs(d))
    rmse_f = np.sqrt(mse_f)
    print("Results ")
    
    print("MAE:", mae_f)
    print("MSE:", mse_f)
    print("RMSE:", rmse_f)
    return mae_f, mse_f, rmse_f

In [11]:
def pdist(sample_1, sample_2, norm=2, eps=1e-5):
    """Compute the matrix of all squared pairwise distances.
    Arguments
    ---------
    sample_1 : torch.Tensor or Variable
        The first sample, should be of shape ``(n_1, d)``.
    sample_2 : torch.Tensor or Variable
        The second sample, should be of shape ``(n_2, d)``.
    norm : float
        The l_p norm to be used.
    Returns
    -------
    torch.Tensor or Variable
        Matrix of shape (n_1, n_2). The [i, j]-th entry is equal to
        ``|| sample_1[i, :] - sample_2[j, :] ||_p``."""
    n_1, n_2 = sample_1.size(0), sample_2.size(0)
    norm = float(norm)
    if norm == 2.:
        norms_1 = torch.sum(sample_1**2, dim=1, keepdim=True)
        norms_2 = torch.sum(sample_2**2, dim=1, keepdim=True)
        norms = (norms_1.expand(n_1, n_2) +
                 norms_2.transpose(0, 1).expand(n_1, n_2))
        distances_squared = norms - 2 * sample_1.mm(sample_2.t())
        return torch.sqrt(eps + torch.abs(distances_squared))
    else:
        dim = sample_1.size(1)
        expanded_1 = sample_1.unsqueeze(1).expand(n_1, n_2, dim)
        expanded_2 = sample_2.unsqueeze(0).expand(n_1, n_2, dim)
        differences = torch.abs(expanded_1 - expanded_2) ** norm
        inner = torch.sum(differences, dim=2, keepdim=False)
        return (eps + inner) ** (1. / norm)

def wasserstein(x,y,p=0.5,lam=10,its=10,sq=False,backpropT=False,cuda=False):
    """return W dist between x and y"""
    '''distance matrix M'''
    nx = x.shape[0]
    ny = y.shape[0]
    
    x = x.squeeze()
    y = y.squeeze()
    
#    pdist = torch.nn.PairwiseDistance(p=2)

    M = pdist(x,y) #distance_matrix(x,y,p=2)
    
    '''estimate lambda and delta'''
    M_mean = torch.mean(M)
    M_drop = F.dropout(M,10.0/(nx*ny))
    delta = torch.max(M_drop).detach()
    eff_lam = (lam/M_mean).detach()

    '''compute new distance matrix'''
    Mt = M
    row = delta*torch.ones(M[0:1,:].shape)
    col = torch.cat([delta*torch.ones(M[:,0:1].shape),torch.zeros((1,1))],0)
    if cuda:
        row = row.cuda()
        col = col.cuda()
        M = M.cuda()
    Mt = torch.cat([M,row],0)
    Mt = torch.cat([Mt,col],1)

    '''compute marginal'''
    a = torch.cat([p*torch.ones((nx,1))/nx,(1-p)*torch.ones((1,1))],0)
    b = torch.cat([(1-p)*torch.ones((ny,1))/ny, p*torch.ones((1,1))],0)

    '''compute kernel'''
    Mlam = eff_lam * Mt
    temp_term = torch.ones(1)*1e-6
    if cuda:
        temp_term = temp_term.cuda()
        a = a.cuda()
        b = b.cuda()
    K = torch.exp(-Mlam) + temp_term
    U = K * Mt
    ainvK = K/a

    u = a

    for i in range(its):
        u = 1.0/(ainvK.matmul(b/torch.t(torch.t(u).matmul(K))))
        if cuda:
            u = u.cuda()
    v = b/(torch.t(torch.t(u).matmul(K)))
    if cuda:
        v = v.cuda()

    upper_t = u*(torch.t(v)*K).detach()

    E = upper_t*Mt
    D = 2*torch.sum(E)

    if cuda:
        D = D.cuda()

    return D, Mlam

In [12]:
reg = 'mine' #'wasserstein' #'mine'
net = 'tarnet'
gpu = 1
# # Define and instantiate the model
 # # Define and instantiate the model
class MLP(nn.Module):
    def __init__(self, dim, hidden_dim):
        super(MLP, self).__init__()
        #hidden_dim = flags.hidden_dim
        hypo_dim = 100
        self.lin1 = nn.Linear(dim, hidden_dim)
        self.lin1_1 = nn.Linear(hidden_dim, hidden_dim)
        self.lin1_2 = nn.Linear(hidden_dim, hypo_dim)
        # tarnet
        if net == 'tarnet':
            self.lin1_3 = nn.Linear(dim, 1)
        else:
            self.lin1_3 = nn.Linear(hypo_dim, 1)

        self.lin_n = nn.Linear(hypo_dim, hypo_dim)
        self.lin_o = nn.Linear(hypo_dim, hypo_dim)

        self.lin2_0 = nn.Linear(hypo_dim, hypo_dim)
        self.lin2_1 = nn.Linear(hypo_dim, hypo_dim)

        self.lin3_0 = nn.Linear(hypo_dim, hypo_dim)
        self.lin3_1 = nn.Linear(hypo_dim, hypo_dim)

        self.lin4_0 = nn.Linear(hypo_dim, 1)
        self.lin4_1 = nn.Linear(hypo_dim, 1)

        self.lin2_0_n = nn.Linear(hypo_dim, hypo_dim)
        self.lin2_1_n = nn.Linear(hypo_dim, hypo_dim)

        self.lin3_0_n = nn.Linear(hypo_dim, hypo_dim)
        self.lin3_1_n = nn.Linear(hypo_dim, hypo_dim)

        self.lin4_0_n = nn.Linear(hypo_dim, 1)
        self.lin4_1_n = nn.Linear(hypo_dim, 1)
        if gpu ==1:
            self.a=torch.nn.Parameter(torch.tensor(1.)).cuda()#.requires_grad_()
            self.b=torch.nn.Parameter(torch.tensor(1.)).cuda()#.requires_grad_()

        else:
            self.a = torch.nn.Parameter(torch.tensor(1.))#.requires_grad_()
            self.b = torch.nn.Parameter(torch.tensor(1.))#.requires_grad_()
        
        self.num_layers = 1
        self.num_directions = 1
        self.lstm = nn.LSTM(hypo_dim, hypo_dim, self.num_layers, batch_first=True)
        self.lstm_n = nn.LSTM(hypo_dim, hypo_dim, self.num_layers, batch_first=True)



        self.lin_mi_0 = nn.Linear(2 *5* hypo_dim, hypo_dim)
        self.lin_mi_1 = nn.Linear(hypo_dim, 1)

        for lin in [self.lin1, self.lin1_1, self.lin1_2, self.lin2_0, self.lin2_1, self.lin1_3, self.lin3_0,
                    self.lin3_1, self.lin4_0, self.lin4_1,  self.lin_mi_0, self.lin_mi_1, self.lin2_0_n,
                    self.lin2_1_n, self.lin3_0_n, self.lin3_1_n, self.lin4_0_n, self.lin4_1_n, self.lin_n,
                    self.lin_o]:
            nn.init.xavier_uniform_(lin.weight)
            nn.init.zeros_(lin.bias)

    def forward(self, input):
        initial = input.view(input.shape)

        x = F.relu(self.lin1(initial))
        x = F.relu(self.lin1_1(x))
        x = F.relu(self.lin1_2(x))
        x = F.relu(x)
        if net == 'tarnet':
            t = self.lin1_3(initial)[:, -1:, :]
        else:
            t = self.lin1_3(x)[:, -1:, :]

        hn_kl = F.relu(self.lin_n(x)) # xn
        ho_kl = F.relu(self.lin_o(x)) # xo
        
        



        loss = torch.tensor(0.)
        #loss = F.kl_div(hn_kl.softmax(dim=-1).log(), ho_kl.softmax(dim=-1), reduction='mean')
        if reg == 'mine':
            
            hn_kl_mi = hn_kl.view([hn_kl.size(0),-1])
            ho_kl_mi = ho_kl.view([ho_kl.size(0),-1])



            ######MINE
            batch_size = hn_kl_mi.size(0)
            tiled_x = torch.cat([hn_kl_mi, hn_kl_mi, ], dim=0)
            idx = torch.randperm(batch_size)

            shuffled_y = ho_kl_mi[idx]
            concat_y = torch.cat([ho_kl_mi, shuffled_y], dim=0)
            inputs = torch.cat([tiled_x, concat_y], dim=1)
            logits = F.relu(self.lin_mi_0(inputs))
            # logits = F.relu(self.lin_mi_1(logits))
            logits = self.lin_mi_1(logits)

            pred_xy = logits[:batch_size]
            pred_x_y = logits[batch_size:]
            
            loss = np.log2(np.exp(1)) * (torch.abs(torch.mean(pred_xy) - torch.log(torch.mean(torch.exp(pred_x_y)))))
            # ######Done MINE
        elif reg == 'wasserstein':
            ###wasserstein
            loss, Mlam = wasserstein(hn_kl, ho_kl, cuda = False)
            loss = -loss
            ###

        
        batch_size, seq_len = hn_kl.shape[0], hn_kl.shape[1]
        hidden_size = 100
        h_0 = torch.randn(self.num_directions * self.num_layers, batch_size, hidden_size).cuda()
        c_0 = torch.randn(self.num_directions * self.num_layers, batch_size, hidden_size).cuda()
        hn_kl,_ = self.lstm_n(hn_kl, (h_0, c_0))
        hn_kl = hn_kl[:, -1:, :] 
        h_0 = torch.randn(self.num_directions * self.num_layers, batch_size, hidden_size).cuda()
        c_0 = torch.randn(self.num_directions * self.num_layers, batch_size, hidden_size).cuda()
        ho_kl,_ = self.lstm(ho_kl, (h_0, c_0))
        ho_kl = ho_kl[:, -1:, :] 
        
        # h1, h2 - different group
        # xn h1_kl
        h_0_n = F.relu(self.lin2_0_n(hn_kl)) # xn
        h_1_n = F.relu(self.lin2_1_n(hn_kl)) # xn

        h_1_n = F.relu(h_1_n)
        h_0_n = F.relu(h_0_n)

        h0_p_n = F.relu(self.lin3_0_n(h_0_n))
        h1_p_n = F.relu(self.lin3_1_n(h_1_n))

        h0_n = self.lin4_0_n(h0_p_n)
        h1_n = self.lin4_1_n(h1_p_n)


        # h1, h2 - different group
        # xn h1_kl
        h_0 = F.relu(self.lin2_0(ho_kl)) # xo
        h_1 = F.relu(self.lin2_1(ho_kl)) # xo

        h_1 = F.relu(h_1)
        h_0 = F.relu(h_0)

        h0_p = F.relu(self.lin3_0(h_0))
        h1_p = F.relu(self.lin3_1(h_1))

        h0 = self.lin4_0(h0_p)
        h1 = self.lin4_1(h1_p)
  
        h0_p =torch.add(h0_p, h0_p_n)
        h1_p =torch.add(h0_p, h1_p_n)

        h0_f = torch.add(self.a*h0, self.b*h0_n)
        h1_f = torch.add(self.a*h0, self.b*h1_n)
        # print("a", self.a)
        # print("b", self.b)

        #loss += torch.tensor(1.) - self.a -self.b
        
        



        return torch.cat((h0_f, h1_f, t), 2) , loss 



In [13]:
def ite(y0_logit, y1_logit):
    y0_pred = torch.sigmoid(y0_logit).float()
    y1_pred = torch.sigmoid(y1_logit).float()
    return y1_pred - y0_pred



In [14]:
# Define loss function helpers
def mean_nll(y_logit, y):
    return nn.functional.binary_cross_entropy_with_logits(y_logit, y.float())

def mean_accuracy(y_logit, y):
    preds = (y_logit > 0.).double()
    return ((preds - y).abs() < 1e-2).float().mean()

In [15]:
gpu = 1
mse = torch.nn.MSELoss()
def penalty(y_logit, y):
    if gpu ==1:
        scale=torch.tensor(1.).cuda().requires_grad_()
    else:
        scale = torch.tensor(1.).requires_grad_()
    loss = mse(y_logit * scale, y)
    grad = autograd.grad(loss, [scale], create_graph=True)[0]
    res = torch.sum(grad ** 2)
    return res


def penalty_coco(y_logit, y):
    if gpu ==1:
        scale = torch.nn.Parameter(torch.normal(1,0.2,[y_logit.shape[1], 1])).cuda().requires_grad_()
    else:
        scale = torch.nn.Parameter(torch.normal(1,0.2,[y_logit.shape[1], 1])).requires_grad_()
    
    scale = scale.float()
    y_logit = y_logit.float()
    y = y.float()
    loss = mse(y_logit @ scale, y)
    grad = autograd.grad(loss, [scale], create_graph=True)[0]

    res = torch.abs(torch.mean((grad**4 *scale)))

    return res

# Proposed

In [25]:
mlp = MLP(13,100).cuda()
lr = 0.001
l2_regularizer_weight = 0.001
optimizer_adam = optim.Adam(mlp.parameters(), lr=lr)
optimizer_sgd = optim.SGD(mlp.parameters(), lr=1e-7, momentum=0.9)

for epoch in range(100):
    #print(epoch)
    loss = 0
    for i in range(len(all_train_loader)):
        for x, y,t in all_train_loader[i]:
            x = x.cuda()
            y = y.cuda()
            t = t.cuda()

            x = x.view([x.size()[0]*x.size()[1],x.size()[2], -1])
            t = t.view([t.size()[0]*t.size()[1], -1,1])
            y = y.view([y.size()[0]*y.size()[1], -1,1])


            logits, mi_loss = mlp(x.float())
            y0_logit = logits[:,:, 0].unsqueeze(2)
            y1_logit = logits[:,:, 1].unsqueeze(2)
            t_logit = logits[:,:, 2].unsqueeze(2)
            y_logit = t * y1_logit + (1 - t) * y0_logit
            
            if gpu == 1:
                weight_norm = torch.tensor(0.).cuda()
            else:
                weight_norm = torch.tensor(0.)
            for w in mlp.parameters():
                weight_norm += w.norm().pow(2)
            
            loss = mean_nll(t_logit, t)
            loss += mse(y_logit, y)
            loss += l2_regularizer_weight * weight_norm
            #print(loss)
            if not torch.isinf(mi_loss).any() and not torch.isnan(mi_loss).any():
                loss += 0.1 * mi_loss
            loss += 0.01*penalty_coco(y_logit, y)


        
            optimizer_adam.zero_grad()
            loss.backward()
            optimizer_adam.step()
    
    

In [26]:
y_all = []
y_pred = []
# ERM_in_mae =[]
# ERM_in_mse = []
# ERM_in_rmse = []
for i in range(len(all_test_loader)):
        for x, y,t in all_test_loader[i]:
            x = x.cuda()
            #y = y.cuda()
            t = t.cuda()
            
            x = x.view([x.size()[0]*x.size()[1],x.size()[2], -1])
            t = t.view([t.size()[0]*t.size()[1], -1,1])
            y = y.view([y.size()[0]*y.size()[1], -1,1])

            #x = x.view([x.size()[0], x.size()[1], -1])
            logits, mi = mlp(x.float())
            #print(logit.shape)
            y0_logit = logits[:,:, 0].unsqueeze(2)
            y1_logit = logits[:,:, 1].unsqueeze(2)
            t_logit = logits[:,:, 2].unsqueeze(2)
            y_logit = t * y1_logit + (1 - t) * y0_logit
            if len(y_all) == 0:
                y_all = np.array(y).reshape(-1)
                y_pred = np.array(y_logit.detach().cpu()).reshape(-1)
            else:
                y_all = np.hstack((y_all, np.array(y).reshape(-1)))
                y_pred = np.hstack((y_pred, np.array(y_logit.detach().cpu()).reshape(-1)))
                
report_res(y_pred, y_all)

Results 
MAE: 0.23508591637230253
MSE: 0.09708651222496997
RMSE: 0.31158708610109304


(0.23508591637230253, 0.09708651222496997, 0.31158708610109304)

In [19]:
y_all = []
y_pred = []
# ERM_in_mae =[]
# ERM_in_mse = []
# ERM_in_rmse = []
for i in range(len(all_test_loader)):
        for x, y,t in all_test_loader[i]:
            x = x.cuda()
            #y = y.cuda()
            t = t.cuda()
            
            x = x.view([x.size()[0]*x.size()[1],x.size()[2], -1])
            t = t.view([t.size()[0]*t.size()[1], -1,1])
            y = y.view([y.size()[0]*y.size()[1], -1,1])

            #x = x.view([x.size()[0], x.size()[1], -1])
            logits, mi = mlp(x.float())
            #print(logit.shape)
            y0_logit = logits[:,:, 0].unsqueeze(2)
            y1_logit = logits[:,:, 1].unsqueeze(2)
            t_logit = logits[:,:, 2].unsqueeze(2)
            y_logit = t * y1_logit + (1 - t) * y0_logit
            if len(y_all) == 0:
                y_all = np.array(y).reshape(-1)
                y_pred = np.array(y_logit.detach().cpu()).reshape(-1)
            else:
                y_all = np.hstack((y_all, np.array(y).reshape(-1)))
                y_pred = np.hstack((y_pred, np.array(y_logit.detach().cpu()).reshape(-1)))
                
report_res(y_pred, y_all)

Results 
MAE: 0.22443563019572813
MSE: 0.09300794632097215
RMSE: 0.30497204186772947


(0.22443563019572813, 0.09300794632097215, 0.30497204186772947)

In [20]:
Pro_out_mae =[]
Pro_out_mse = []
Pro_out_rmse = []
for i in range(len(all_test_in_loader)):
    y_all = []
    y_pred = []
    for x, y,t in all_test_in_loader[i]:
        x = x.cuda()
        #y = y.cuda()
        t = t.cuda()

        x = x.view([x.size()[0]*x.size()[1],x.size()[2], -1])
        t = t.view([t.size()[0]*t.size()[1], -1,1])
        y = y.view([y.size()[0]*y.size()[1], -1,1])

        #x = x.view([x.size()[0], x.size()[1], -1])
        logits, mi = mlp(x.float())
        #print(logit.shape)
        y0_logit = logits[:,:, 0].unsqueeze(2)
        y1_logit = logits[:,:, 1].unsqueeze(2)
        t_logit = logits[:,:, 2].unsqueeze(2)
        y_logit = t * y1_logit + (1 - t) * y0_logit
        if len(y_all) == 0:
            y_all = np.array(y).reshape(-1)
            y_pred = np.array(y_logit.detach().cpu()).reshape(-1)
        else:
            y_all = np.hstack((y_all, np.array(y).reshape(-1)))
            y_pred = np.hstack((y_pred, np.array(y_logit.detach().cpu()).reshape(-1)))
                
    mae_f, mse_f, rmse_f = report_res(y_pred, y_all)
    Pro_out_mse.append(mse_f)
    Pro_out_mae.append(mae_f)
    Pro_out_rmse.append(rmse_f)


Results 
MAE: 0.19079634337671103
MSE: 0.05646493558114684
RMSE: 0.2376235164733214
Results 
MAE: 0.2258738535195262
MSE: 0.08425541642115372
RMSE: 0.2902678356641564
Results 
MAE: 0.20748424019971112
MSE: 0.08890331848637284
RMSE: 0.2981665951886174
Results 
MAE: 0.22376905021378732
MSE: 0.09310422843034435
RMSE: 0.3051298550295339
Results 
MAE: 0.21323903587967846
MSE: 0.08515539679746284
RMSE: 0.2918139763573068
Results 
MAE: 0.1789755952688881
MSE: 0.056674143043701004
RMSE: 0.23806331729962305
Results 
MAE: 0.2139190831844433
MSE: 0.08651345740936185
RMSE: 0.2941317007895644


In [21]:
y_all = []
y_pred = []
for i in range(len(all_test_in_loader)):
        for x, y,t in all_test_in_loader[i]:
            x = x.cuda()
            #y = y.cuda()
            t = t.cuda()
            
            x = x.view([x.size()[0]*x.size()[1],x.size()[2], -1])
            t = t.view([t.size()[0]*t.size()[1], -1,1])
            y = y.view([y.size()[0]*y.size()[1], -1,1])

            #x = x.view([x.size()[0], x.size()[1], -1])
            logits, mi = mlp(x.float())
            #print(logit.shape)
            y0_logit = logits[:,:, 0].unsqueeze(2)
            y1_logit = logits[:,:, 1].unsqueeze(2)
            t_logit = logits[:,:, 2].unsqueeze(2)
            y_logit = t * y1_logit + (1 - t) * y0_logit
            if len(y_all) == 0:
                y_all = np.array(y).reshape(-1)
                y_pred = np.array(y_logit.detach().cpu()).reshape(-1)
            else:
                y_all = np.hstack((y_all, np.array(y).reshape(-1)))
                y_pred = np.hstack((y_pred, np.array(y_logit.detach().cpu()).reshape(-1)))
                
report_res(y_pred, y_all)
            #print(y_logit.shape)

Results 
MAE: 0.21086974615804555
MSE: 0.07931272167603294
RMSE: 0.2816251438988233


(0.21086974615804555, 0.07931272167603294, 0.2816251438988233)

In [22]:
Pro_in_mae =[]
Pro_in_mse = []
Pro_in_rmse = []
for i in range(len(all_test_in_loader)):
    y_all = []
    y_pred = []
    for x, y,t in all_test_in_loader[i]:
        x = x.cuda()
        #y = y.cuda()
        t = t.cuda()

        x = x.view([x.size()[0]*x.size()[1],x.size()[2], -1])
        t = t.view([t.size()[0]*t.size()[1], -1,1])
        y = y.view([y.size()[0]*y.size()[1], -1,1])

        #x = x.view([x.size()[0], x.size()[1], -1])
        logits, mi = mlp(x.float())
        #print(logit.shape)
        y0_logit = logits[:,:, 0].unsqueeze(2)
        y1_logit = logits[:,:, 1].unsqueeze(2)
        t_logit = logits[:,:, 2].unsqueeze(2)
        y_logit = t * y1_logit + (1 - t) * y0_logit
        if len(y_all) == 0:
            y_all = np.array(y).reshape(-1)
            y_pred = np.array(y_logit.detach().cpu()).reshape(-1)
        else:
            y_all = np.hstack((y_all, np.array(y).reshape(-1)))
            y_pred = np.hstack((y_pred, np.array(y_logit.detach().cpu()).reshape(-1)))
                
    mae_f, mse_f, rmse_f = report_res(y_pred, y_all)
    Pro_in_mse.append(mse_f)
    Pro_in_mae.append(mae_f)
    Pro_in_rmse.append(rmse_f)
        

Results 
MAE: 0.19041473082269966
MSE: 0.05623527870588155
RMSE: 0.23713978726877855
Results 
MAE: 0.225091332103704
MSE: 0.08417954671463351
RMSE: 0.29013711709230433
Results 
MAE: 0.20757464079087518
MSE: 0.08946155844498142
RMSE: 0.2991012511591709
Results 
MAE: 0.22546348503268024
MSE: 0.094589867302112
RMSE: 0.307554657422241
Results 
MAE: 0.21286762215649638
MSE: 0.08496595679322183
RMSE: 0.2914892052773513
Results 
MAE: 0.18051344175248626
MSE: 0.05732322674349212
RMSE: 0.23942269471270286
Results 
MAE: 0.21423055054732532
MSE: 0.0855584269765025
RMSE: 0.29250372130368274


In [35]:
all_test_in_loader = []
for key in train_envs.keys():
    dataset_train = Dataset_in_test(train_envs[key]['previous_covariates'], train_envs[key]['outcomes'],train_envs[key]['previous_treatments'], T=5)
    train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=4, shuffle=True)  
    all_test_in_loader.append(train_loader)

In [36]:
y_all = []
y_pred = []
for i in range(len(all_test_in_loader)):
        for x, y,t in all_test_in_loader[i]:
            x = x.cuda()
            #y = y.cuda()
            t = t.cuda()
            x = x.view([x.size()[0]*x.size()[1],x.size()[2], -1])
            t = t.view([t.size()[0]*t.size()[1], -1,1])
            y = y.view([y.size()[0]*y.size()[1], -1,1])

            #x = x.view([x.size()[0], x.size()[1], -1])
            logits, mi_loss = mlp(x.float())
            #print(logit.shape)
            y0_logit = logits[:,:, 0].unsqueeze(2)
            y1_logit = logits[:,:, 1].unsqueeze(2)
            t_logit = logits[:,:, 2].unsqueeze(2)
            y_logit = t * y1_logit + (1 - t) * y0_logit
            if len(y_all) == 0:
                y_all = np.array(y).reshape(-1)
                y_pred = np.array(y_logit.detach().cpu()).reshape(-1)
            else:
                y_all = np.hstack((y_all, np.array(y).reshape(-1)))
                y_pred = np.hstack((y_pred, np.array(y_logit.detach().cpu()).reshape(-1)))
                



In [37]:
report_res(y_pred, y_all)

Results 
MAE: 0.22874074805136743
MSE: 0.08574023929682352
RMSE: 0.29281434271022916


(0.22874074805136743, 0.08574023929682352, 0.29281434271022916)

In [110]:
mlp = MLP(13,100).cuda()
lr = 0.001
l2_regularizer_weight = 0.001
optimizer_adam = optim.Adam(mlp.parameters(), lr=lr)
optimizer_sgd = optim.SGD(mlp.parameters(), lr=1e-7, momentum=0.9)

for epoch in range(100):
    #print(epoch)
    loss = 0
    for i in range(len(all_train_loader)):
        for x, y,t in all_train_loader[i]:
            x = x.cuda()
            y = y.cuda()
            t = t.cuda()

            x = x.view([x.size()[0]*x.size()[1],x.size()[2], -1])
            t = t.view([t.size()[0]*t.size()[1], -1,1])
            y = y.view([y.size()[0]*y.size()[1], -1,1])


            logits, mi_loss = mlp(x.float())
            #print(logit.shape)
            y0_logit = logits[:,:, 0].unsqueeze(2)
            y1_logit = logits[:,:, 1].unsqueeze(2)
            t_logit = logits[:,:, 2].unsqueeze(2)
#             print(t.shape)
#             print(y1_logit.shape)
            y_logit = t * y1_logit + (1 - t) * y0_logit
            
            if gpu == 1:
                weight_norm = torch.tensor(0.).cuda()
            else:
                weight_norm = torch.tensor(0.)
            for w in mlp.parameters():
                weight_norm += w.norm().pow(2)
            
            loss += mean_nll(t_logit, t)
            loss += mse(y_logit, y)
            loss += l2_regularizer_weight * weight_norm
            #print(loss)
            if not torch.isinf(mi_loss).any() and not torch.isnan(mi_loss).any():
                #print(mi_loss)
                loss = 0.1 * mi_loss
            loss += 0.01*penalty_coco(y_logit, y)
        #print(i, ite(y0_logit, y1_logit).mean())
        #print(i, mse(y_logit, y).mean())


        
    optimizer_adam.zero_grad()
    loss.backward()
    optimizer_adam.step()
    
    

In [111]:
y_all = []
y_pred = []
for i in range(len(all_test_loader)):
        for x, y,t in all_test_loader[i]:
            x = x.cuda()
            #y = y.cuda()
            t = t.cuda()
            
            x = x.view([x.size()[0]*x.size()[1],x.size()[2], -1])
            t = t.view([t.size()[0]*t.size()[1], -1,1])
            y = y.view([y.size()[0]*y.size()[1], -1,1])

            #x = x.view([x.size()[0], x.size()[1], -1])
            logits, mi_loss = mlp(x.float())
            #print(logit.shape)
            y0_logit = logits[:,:, 0].unsqueeze(2)
            y1_logit = logits[:,:, 1].unsqueeze(2)
            t_logit = logits[:,:, 2].unsqueeze(2)
            y_logit = t * y1_logit + (1 - t) * y0_logit
            if len(y_all) == 0:
                y_all = np.array(y).reshape(-1)
                y_pred = np.array(y_logit.detach().cpu()).reshape(-1)
            else:
                y_all = np.hstack((y_all, np.array(y).reshape(-1)))
                y_pred = np.hstack((y_pred, np.array(y_logit.detach().cpu()).reshape(-1)))

report_res(y_pred, y_all)               

Results 
MAE: 0.29308792773010195
MSE: 0.19206802748397342
RMSE: 0.43825566452012166


In [112]:
y_all = []
y_pred = []
for i in range(len(all_test_in_loader)):
        for x, y,t in all_test_in_loader[i]:
            x = x.cuda()
            #y = y.cuda()
            t = t.cuda()
            x = x.view([x.size()[0]*x.size()[1],x.size()[2], -1])
            t = t.view([t.size()[0]*t.size()[1], -1,1])
            y = y.view([y.size()[0]*y.size()[1], -1,1])

            #x = x.view([x.size()[0], x.size()[1], -1])
            logits, mi_loss = mlp(x.float())
            #print(logit.shape)
            y0_logit = logits[:,:, 0].unsqueeze(2)
            y1_logit = logits[:,:, 1].unsqueeze(2)
            t_logit = logits[:,:, 2].unsqueeze(2)
            y_logit = t * y1_logit + (1 - t) * y0_logit
            if len(y_all) == 0:
                y_all = np.array(y).reshape(-1)
                y_pred = np.array(y_logit.detach().cpu()).reshape(-1)
            else:
                y_all = np.hstack((y_all, np.array(y).reshape(-1)))
                y_pred = np.hstack((y_pred, np.array(y_logit.detach().cpu()).reshape(-1)))
                
report_res(y_pred, y_all)  
                

Results 
MAE: 0.2513970825176546
MSE: 0.14099167850264693
RMSE: 0.37548858638132654
